# Train Type 1

TRAIN CLASS 0: collect the 55k fakes already generated

In [16]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('C:/Users/nello/Documents/vscode_projects/Thesis/3_image_generation/generated_images/csv/train_synthetics.csv')

# Filter rows where '6_way_label' equals 0 and 'fake_id' starts with 'SD', 'DL', or 'GL'
filtered_rows_SD = df[(df['6_way_label'] == 0) & (df['fake_id'].str.startswith('SD'))]
filtered_rows_DL = df[(df['6_way_label'] == 0) & (df['fake_id'].str.startswith('DL'))]
filtered_rows_GL = df[(df['6_way_label'] == 0) & (df['fake_id'].str.startswith('GL'))]

# Concatenate columns for all groups
result = pd.concat([filtered_rows_SD, filtered_rows_DL, filtered_rows_GL])
print(len(result))

# Save the result to a new CSV file
result.to_csv('csv/train_class_0_already_generated.csv', index=False)

print("Filtered IDs saved to train_class_0_already_generated.csv")

55610
Filtered IDs saved to train_class_0_already_generated.csv


save a csv for the 107k pristine and the 107k-55k to be generated

In [17]:
import pandas as pd

MULTIMODAL_TRAIN_CLEANED_WITH_CLASS_TSV = "C:/Users/nello/OneDrive - University of Pisa/TESI/TSV_JSON/1_dataset_cleaning/tsv/train_tsv_with_class.tsv"
ALREADY_GENERATED_FROM_CLASS_0 = "csv/train_class_0_already_generated.csv"

train = pd.read_csv(MULTIMODAL_TRAIN_CLEANED_WITH_CLASS_TSV, sep='\t')
train_filtered = train[train['6_way_label'] == 0]

train_class_0_already_generated = pd.read_csv(ALREADY_GENERATED_FROM_CLASS_0)
print("len of train_class_0_already_generated: "+str(len(train_class_0_already_generated)))

# Filter rows from train DataFrame that are not in the 'id' column of the other DataFrame
train_not_in_other = train_filtered[~train_filtered['id'].isin(train_class_0_already_generated['id'])]

# Select the first 107,549 rows from train_not_in_other
train_pristine = train_not_in_other.head(107548)

# Save train_pristine to CSV
print("len of train_class_0_pristine: "+str(len(train_pristine)))
train_pristine.to_csv('csv/train_class_0_pristine.csv', index=False)

# Filter rows from train_filtered that are not in train_class_0_already_generated or train_pristine
train_remaining = train_filtered[~train_filtered['id'].isin(train_class_0_already_generated['id']) & ~train_filtered['id'].isin(train_pristine['id'])]

# Save the remaining rows to a new CSV file
print("len of train_class_0_to_be_generated: "+str(len(train_remaining)))
train_remaining.to_csv('csv/train_class_0_to_be_generated.csv', index=False)
print("train_class_0_already_generated: 55.610 + train_class_0_to_be_generated: 51.938 = " + str(55610+51938))

len of train_class_0_already_generated: 55610
len of train_pristine: 107548
len of train_class_0_to_be_generated: 51939
train_class_0_already_generated: 55.610 + train_class_0_to_be_generated: 51.938 = 107548


rename the columns real_image and fake_image to pristine_image and generated_image inside train_class_0_already_generated.csv

In [20]:
df = pd.read_csv('csv/train_class_0_already_generated.csv')

# Rename the column
df.rename(columns={"real_image": "pristine_image"}, inplace=True)
df.rename(columns={"fake_image": "generated_image"}, inplace=True)
df.to_csv("csv/train_class_0_already_generated.csv", index=False)

also change the value of columns real_image and fake_image in train_class_0_already_generated.csv from real_image=1 and fake_image=0 to real_image=0 and fake_image=1

In [21]:
df = pd.read_csv('csv/train_class_0_already_generated.csv')

# Swap the values of the two columns
df['pristine_image'], df['generated_image'] = df['generated_image'], df['pristine_image']

df.to_csv("csv/train_class_0_already_generated.csv", index=False)

reduce columns of train_class_0_to_be_generated and train_class_0_pristine csv:

In [22]:
import pandas as pd

df = pd.read_csv('csv/train_class_0_to_be_generated.csv')

# Rename the column
df.rename(columns={"clean_title": "original_caption"}, inplace=True)

df['pristine_image'] = 1
df['generated_image'] = 0
df['real_text'] = 1
df['fakenews_text'] = 0

# Select only the desired columns in the required order
desired_columns = ['id', 'author', 'original_caption', 'num_comments', '6_way_label', 'pristine_image', 'generated_image', 'real_text', 'fakenews_text']
df = df[desired_columns]

df.to_csv("csv/train_class_0_to_be_generated.csv", index=False)

In [23]:
import pandas as pd

df = pd.read_csv('csv/train_class_0_pristine.csv')

# Rename the column
df.rename(columns={"clean_title": "original_caption"}, inplace=True)

df['pristine_image'] = 1
df['generated_image'] = 0
df['real_text'] = 1
df['fakenews_text'] = 0

# Select only the desired columns in the required order
desired_columns = ['id', 'author', 'original_caption', 'num_comments', '6_way_label', 'pristine_image', 'generated_image', 'real_text', 'fakenews_text']
df = df[desired_columns]

df.to_csv("csv/train_class_0_pristine.csv", index=False)

check if every id inside the 250k train_tsv_with_class.csv is contained in the union of the 3 csv train_pristine + train_class_0_already_generated + train_class_0_to_be_generated

In [35]:
import pandas as pd

# Load the 'id' column from the first three CSV files into sets
csv1_ids = set(pd.read_csv("csv/train_pristine.csv")['id'])
csv2_ids = set(pd.read_csv("csv/train_class_0_to_be_generated.csv")['id'])
csv3_ids = set(pd.read_csv("csv/train_class_0_already_generated.csv")['id'])

# Find the union of these three sets
union_ids = csv1_ids.union(csv2_ids, csv3_ids)

csv4_ids = set(pd.read_csv("C:/Users/nello/OneDrive - University of Pisa/TESI/TSV_JSON/1_dataset_cleaning/tsv/train_tsv_with_class.tsv", sep='\t').loc[pd.read_csv("C:/Users/nello/OneDrive - University of Pisa/TESI/TSV_JSON/1_dataset_cleaning/tsv/train_tsv_with_class.tsv", sep='\t')['6_way_label'] == 0, 'id'])

print(len(union_ids))
print(len(csv4_ids))

# Check if the union_ids set is equal to the csv4_ids set
if union_ids == csv4_ids:
    print("The union of IDs from the first three CSVs corresponds exactly to the IDs from train_tsv_with_class.csv where '6_way_label' = 0.")
else:
    print("The union of IDs from the first three CSVs does not correspond exactly to the IDs from train_tsv_with_class.csv where '6_way_label' = 0.")

215097
215097
The union of IDs from the first three CSVs corresponds exactly to the IDs from train_tsv_with_class.csv where '6_way_label' = 0.


Now i print out how many StableDiffusion, Dreamlike and Glide images i have already generated:

In [26]:
df = pd.read_csv("csv/train_class_0_already_generated.csv")

# Count the rows where the 'fake_id' column starts with 'SD', 'DL', or 'GL'
count_SD = len(df[df['fake_id'].str.startswith('SD')])
count_DL = len(df[df['fake_id'].str.startswith('DL')])
count_GL = len(df[df['fake_id'].str.startswith('GL')])

print("Number of rows where 'fake_id' starts with 'SD':", count_SD)
print("Number of rows where 'fake_id' starts with 'DL':", count_DL)
print("Number of rows where 'fake_id' starts with 'GL':", count_GL)

Number of rows where 'fake_id' starts with 'SD': 22350
Number of rows where 'fake_id' starts with 'DL': 22044
Number of rows where 'fake_id' starts with 'GL': 11216


Thus, to get to 107.549 generated images i want to generate other:

- SD: 40.000 - 22.350 = 17.650
- DL: 40.000 - 22.044 = 17.956
- GL: (107.549 - 80.000) - 11.216 = 16.333

so 51.939 in total from the train_class_0_to_be_generated.csv, but first i need to caption all the images from train_class_0_to_be_generated.csv

Here we check that every pristine image inside train_class_0_to_be_generated csv is already in the image folder

In [13]:
import os
import pandas as pd

# Load the 'id' column from the CSV file into a set
csv_ids = set(pd.read_csv("csv/train/train_class_0_to_be_generated.csv")['id'])

# Get a list of all the image filenames in the folder
folder_path = "/home/enriconello/DeepFakeDetection/dataset"
folder_images = set(filename.split('.')[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = csv_ids.intersection(folder_images)

# Count the number of matching images
num_matching_images = len(matching_images)

print("Number of matching images:", num_matching_images)

Number of matching images: 51939


Here we check that every pristine image inside train_class_0_pristine csv is already in the image folder

In [14]:
import os
import pandas as pd

# Load the 'id' column from the CSV file into a set
csv_ids = set(pd.read_csv("csv/train/train_class_0_pristine.csv")['id'])

# Get a list of all the image filenames in the folder
folder_path = "/home/enriconello/DeepFakeDetection/dataset"
folder_images = set(filename.split('.')[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = csv_ids.intersection(folder_images)

# Count the number of matching images
num_matching_images = len(matching_images)

print("Number of matching images:", num_matching_images)

Number of matching images: 107548


Here we check that every pristine image inside train_class_0_already_generated csv is already in the image folder

In [19]:
import os
import pandas as pd

# Load the 'id' column from the CSV file into a set
csv_ids = set(pd.read_csv("csv/train/train_class_0_already_generated.csv")['fake_id'])

# Get a list of all the image filenames in the folder
folder_path = "/home/enriconello/DeepFakeDetection/dataset"
folder_images = set(filename.split('.')[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = csv_ids.intersection(folder_images)

# Count the number of matching images
num_matching_images = len(matching_images)

print("Number of matching images:", num_matching_images)

Number of matching images: 55610


-------------------------

# Train Type 2

Now construct the train "type 2" part, thus the 85k pristine with 6_way_label == 1,2,3,5 (4 manipulated deleted) and 85k generated still with 6_way_label == 1,2,3,5

In [2]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('C:/Users/nello/Documents/vscode_projects/Thesis/3_image_generation/generated_images/csv/train_synthetics.csv')

# Filter rows where '6_way_label' equals 1, 2, 3, or 5 and 'fake_id' starts with 'SD', 'DL', or 'GL'
filtered_rows = df[(df['6_way_label'].isin([1, 2, 3, 5])) & (df['fake_id'].str.startswith(('SD', 'DL', 'GL')))]

# Save the result to a new CSV file
print(len(filtered_rows))
filtered_rows.to_csv('csv/train/train_class_1_2_3_5_already_generated.csv', index=False)

44390


save a csv for the 85k pristine and the 85k-44k to be generated

In [3]:
import pandas as pd

MULTIMODAL_TRAIN_CLEANED_WITH_CLASS_TSV = "C:/Users/nello/OneDrive - University of Pisa/TESI/TSV_JSON/1_dataset_cleaning/tsv/train_tsv_with_class.tsv"
ALREADY_GENERATED_FROM_CLASS_1_2_3_5 = "csv/train/train_class_1_2_3_5_already_generated.csv"

train = pd.read_csv(MULTIMODAL_TRAIN_CLEANED_WITH_CLASS_TSV, sep='/t')
train_filtered = train[train['6_way_label'].isin([1, 2, 3, 5])]

train_class_0_already_generated = pd.read_csv(ALREADY_GENERATED_FROM_CLASS_1_2_3_5)
print("len of train_class_1_2_3_5_already_generated: "+str(len(train_class_0_already_generated)))

# Filter rows from train DataFrame that are not in the 'id' column of the other DataFrame
train_not_in_other = train_filtered[~train_filtered['id'].isin(train_class_0_already_generated['id'])]

# Select the first 107,549 rows from train_not_in_other
train_pristine = train_not_in_other.head(85387)

# Save train_pristine to CSV
print("len of train_class_1_2_3_5_pristine: "+str(len(train_pristine)))
train_pristine.to_csv('csv/train/train_class_1_2_3_5_pristine.csv', index=False)

# Filter rows from train_filtered that are not in train_class_0_already_generated or train_pristine
train_remaining = train_filtered[~train_filtered['id'].isin(train_class_0_already_generated['id']) & ~train_filtered['id'].isin(train_pristine['id'])]

# Save the remaining rows to a new CSV file
print("len of train_class_1_2_3_5_to_be_generated: "+str(len(train_remaining)))
train_remaining.to_csv('csv/train/train_class_1_2_3_5_to_be_generated.csv', index=False)
print("train_class_1_2_3_5_already_generated: 44.390 + train_class_1_2_3_5_to_be_generated: 40.997 = " + str(44390+40997))

len of train_class_1_2_3_5_already_generated: 44390
len of train_class_1_2_3_5_pristine: 85387
len of train_class_1_2_3_5_to_be_generated: 40997
train_class_1_2_3_5_already_generated: 44.390 + train_class_1_2_3_5_to_be_generated: 40.997 = 85387


rename the columns real_image and fake_image to pristine_image and generated_image inside train_class_1_2_3_5_already_generated.csv

In [4]:
df = pd.read_csv('csv/train/train_class_1_2_3_5_already_generated.csv')

# Rename the column
df.rename(columns={"real_image": "pristine_image"}, inplace=True)
df.rename(columns={"fake_image": "generated_image"}, inplace=True)
df.to_csv("csv/train/train_class_1_2_3_5_already_generated.csv", index=False)

also change the value of columns real_image and fake_image in train_class_1_2_3_5_already_generated.csv from real_image=1 and fake_image=0 to real_image=0 and fake_image=1

In [5]:
df = pd.read_csv('csv/train/train_class_1_2_3_5_already_generated.csv')

# Swap the values of the two columns
df['pristine_image'], df['generated_image'] = df['generated_image'], df['pristine_image']

df.to_csv("csv/train/train_class_1_2_3_5_already_generated.csv", index=False)

reduce columns of train_class_1_2_3_5_to_be_generated and train_class_1_2_3_5_pristine csv:

In [6]:
import pandas as pd

df = pd.read_csv('csv/train/train_class_1_2_3_5_to_be_generated.csv')

# Rename the column
df.rename(columns={"clean_title": "original_caption"}, inplace=True)

df['pristine_image'] = 1
df['generated_image'] = 0
df['real_text'] = 0
df['fakenews_text'] = 1

# Select only the desired columns in the required order
desired_columns = ['id', 'author', 'original_caption', 'num_comments', '6_way_label', 'pristine_image', 'generated_image', 'real_text', 'fakenews_text']
df = df[desired_columns]

df.to_csv("csv/train/train_class_1_2_3_5_to_be_generated.csv", index=False)

In [7]:
import pandas as pd

df = pd.read_csv('csv/train/train_class_1_2_3_5_pristine.csv')

# Rename the column
df.rename(columns={"clean_title": "original_caption"}, inplace=True)

df['pristine_image'] = 1
df['generated_image'] = 0
df['real_text'] = 0
df['fakenews_text'] = 1

# Select only the desired columns in the required order
desired_columns = ['id', 'author', 'original_caption', 'num_comments', '6_way_label', 'pristine_image', 'generated_image', 'real_text', 'fakenews_text']
df = df[desired_columns]

df.to_csv("csv/train/train_class_1_2_3_5_pristine.csv", index=False)

Show the distribution of the original classes (1,2,3,5) for the csv

In [8]:
import pandas as pd

df = pd.read_csv('csv/train/train_class_1_2_3_5_pristine.csv')

# Count the number of rows for each value of '6_way_label'
label_counts = df['6_way_label'].value_counts()

# Print the counts for each unique value of '6_way_label'
print("Counts for each unique value of '6_way_label':")
print(label_counts)

Counts for each unique value of '6_way_label':
6_way_label
2    52898
1    16290
5    10484
3     5715
Name: count, dtype: int64


In [9]:
df = pd.read_csv('csv/train/train_class_1_2_3_5_already_generated.csv')

# Count the number of rows for each value of '6_way_label'
label_counts = df['6_way_label'].value_counts()

# Print the counts for each unique value of '6_way_label'
print("Counts for each unique value of '6_way_label':")
print(label_counts)

Counts for each unique value of '6_way_label':
6_way_label
2    27339
1     8566
5     5565
3     2920
Name: count, dtype: int64


In [11]:
import pandas as pd

df = pd.read_csv('csv/train/train_class_1_2_3_5_already_generated.csv')

# Group the DataFrame by '6_way_label'
grouped = df.groupby('6_way_label')

# Iterate over each group
for label, group in grouped:
    # Filter the group where 'id' column starts with "SD", "GL", or "DL"
    sd_count = len(group[group['fake_id'].str.startswith('SD')])
    gl_count = len(group[group['fake_id'].str.startswith('GL')])
    dl_count = len(group[group['fake_id'].str.startswith('DL')])
    
    # Print counts for each value of '6_way_label'
    print(f"Counts for '6_way_label' {label}:")
    print(f"Starts with 'SD': {sd_count}")
    print(f"Starts with 'GL': {gl_count}")
    print(f"Starts with 'DL': {dl_count}")
    print()


Counts for '6_way_label' 1:
Starts with 'SD': 3423
Starts with 'GL': 1681
Starts with 'DL': 3462

Counts for '6_way_label' 2:
Starts with 'SD': 10887
Starts with 'GL': 5429
Starts with 'DL': 11023

Counts for '6_way_label' 3:
Starts with 'SD': 1125
Starts with 'GL': 588
Starts with 'DL': 1207

Counts for '6_way_label' 5:
Starts with 'SD': 2215
Starts with 'GL': 1086
Starts with 'DL': 2264



In [13]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('csv/train/train_class_1_2_3_5_to_be_generated.csv')

# Define the start and end indices for each interval
intervals = {
    "Generated with SD": (0, 12350),
    "Generated with DL": (12350, 24394),
    "Generated with GL": (24394, len(df))
}

# Iterate over each interval
for method, (start_index, end_index) in intervals.items():
    # Filter the DataFrame based on the interval
    interval_df = df.iloc[start_index:end_index]
    
    # Group the interval DataFrame by '6_way_label'
    grouped = interval_df.groupby('6_way_label')
    
    # Print the count of rows with each '6_way_label' value within the interval
    print(f"{method}:")
    for label, group in grouped:
        label_count = len(group)
        print(f"6_way_label: {label} in the interval [{start_index}, {end_index}): {label_count}")
    print()

Generated with SD:
6_way_label: 1 in the interval [0, 12350): 2356
6_way_label: 2 in the interval [0, 12350): 7614
6_way_label: 3 in the interval [0, 12350): 860
6_way_label: 5 in the interval [0, 12350): 1520

Generated with DL:
6_way_label: 1 in the interval [12350, 24394): 2315
6_way_label: 2 in the interval [12350, 24394): 7437
6_way_label: 3 in the interval [12350, 24394): 806
6_way_label: 5 in the interval [12350, 24394): 1486

Generated with GL:
6_way_label: 1 in the interval [24394, 40997): 3146
6_way_label: 2 in the interval [24394, 40997): 10315
6_way_label: 3 in the interval [24394, 40997): 1080
6_way_label: 5 in the interval [24394, 40997): 2062



check if every id inside the train_tsv_with_class.csv with '6_way_label' = 1,2,3,5 is contained in the union of the 3 csv train_class_1_2_3_5_pristine + train_class_1_2_3_5_already_generated + train_class_1_2_3_5_to_be_generated

In [12]:
import pandas as pd

# Load the 'id' column from the first three CSV files into sets
csv1_ids = set(pd.read_csv("csv/train/train_class_1_2_3_5_pristine.csv")['id'])
csv2_ids = set(pd.read_csv("csv/train/train_class_1_2_3_5_to_be_generated.csv")['id'])
csv3_ids = set(pd.read_csv("csv/train/train_class_1_2_3_5_already_generated.csv")['id'])

# Find the union of these three sets
union_ids = csv1_ids.union(csv2_ids, csv3_ids)

csv4_ids = set(pd.read_csv("C:/Users/nello/OneDrive - University of Pisa/TESI/TSV_JSON/1_dataset_cleaning/tsv/train_tsv_with_class.tsv", sep='\t').loc[pd.read_csv("C:/Users/nello/OneDrive - University of Pisa/TESI/TSV_JSON/1_dataset_cleaning/tsv/train_tsv_with_class.tsv", sep='\t')['6_way_label'].isin([1, 2, 3, 5]), 'id'])

print(len(union_ids))
print(len(csv4_ids))

# Check if the union_ids set is equal to the csv4_ids set
if union_ids == csv4_ids:
    print("The union of IDs from the first three CSVs corresponds exactly to the IDs from train_tsv_with_class.csv where '6_way_label' = 1,2,3,5.")
else:
    print("The union of IDs from the first three CSVs does not correspond exactly to the IDs from train_tsv_with_class.csv where '6_way_label' = 1,2,3,5.")

170774
170774
The union of IDs from the first three CSVs corresponds exactly to the IDs from train_tsv_with_class.csv where '6_way_label' = 1,2,3,5.


Thus, to get to 85.387 generated images i want to generate other 40.997:

- SD: 30.000 - 17.650 = 12.350
- DL: 30.000 - 17.956 = 12.044
- GL: (85.387 - 60.000) - 8.784 = 16.603

so 40.997 in total from the train_class_1_2_3_5_to_be_generated.csv, but first i need to caption all the images from train_class_1_2_3_5_to_be_generated.csv

Here we check that every pristine image inside train_class_1_2_3_5_to_be_generated csv is already in the image folder

In [2]:
import os
import pandas as pd

# Load the 'id' column from the CSV file into a set
csv_ids = set(pd.read_csv("csv/train/train_class_1_2_3_5_to_be_generated.csv")['id'])

# Get a list of all the image filenames in the folder
folder_path = "C:/Users/nello/Desktop/TESI/dataset_after_merging_WITH_DUPLICATES"
folder_images = set(filename.split('.')[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = csv_ids.intersection(folder_images)

# Count the number of matching images
num_matching_images = len(matching_images)

print("Number of matching images:", num_matching_images)

Number of matching images: 40997


Here we check that every pristine image inside train_class_1_2_3_5_pristine csv is already in the image folder

In [18]:
import os
import pandas as pd

# Load the 'id' column from the CSV file into a set
csv_ids = set(pd.read_csv("csv/train/train_class_1_2_3_5_pristine.csv")['id'])

# Get a list of all the image filenames in the folder
folder_path = "/home/enriconello/DeepFakeDetection/dataset"
folder_images = set(filename.split('.')[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = csv_ids.intersection(folder_images)

# Count the number of matching images
num_matching_images = len(matching_images)

print("Number of matching images:", num_matching_images)

Number of matching images: 85387


Here we check that every pristine image inside train_class_1_2_3_5_already_generated csv is already in the image folder

In [17]:
import os
import pandas as pd

# Load the 'id' column from the CSV file into a set
csv_ids = set(pd.read_csv("csv/train/train_class_1_2_3_5_already_generated.csv")['fake_id'])

# Get a list of all the image filenames in the folder
folder_path = "/home/enriconello/DeepFakeDetection/dataset"
folder_images = set(filename.split('.')[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = csv_ids.intersection(folder_images)

# Count the number of matching images
num_matching_images = len(matching_images)

print("Number of matching images:", num_matching_images)

Number of matching images: 44390


Clean the dataset folder from Manipulated Images (class 4)

In [4]:
import os
import pandas as pd

csv_files = ['/home/enriconello/DeepFakeDetection/Thesis/3_image_generation/generated_images/csv/train.csv',
             '/home/enriconello/DeepFakeDetection/Thesis/3_image_generation/generated_images/csv/test_with_duplicates.csv',
             '/home/enriconello/DeepFakeDetection/Thesis/3_image_generation/generated_images/csv/validation_with_duplicates.csv']

# image folder path 
image_folder = '/home/enriconello/DeepFakeDetection/dataset'

unique_ids = set()

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    
    # Filter rows where '6_way_label' equals to the class Manipulated images
    filtered_df = df[df['6_way_label'] == 4]
    unique_ids.update(filtered_df['id'])

num_images_removed = 0

for id_value in unique_ids:
    image_path = os.path.join(image_folder, f"{id_value}.jpg")
    
    if os.path.exists(image_path):
        os.remove(image_path)
        num_images_removed += 1
    else:
        continue

print(f"Total number of images removed: {num_images_removed}")

Total number of images removed: 190185


Clean the train_class_1_2_3_5_captioned csv, deleting the generated captions that starts with "araf" (buggy captions)

In [9]:
import pandas as pd

original_df = pd.read_csv('/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/train/train_class_1_2_3_5_captioned.csv')

# Create a copy of the original DataFrame
modified_df = original_df.copy()

# Remove words starting with "araf" in the "generated_caption" column
modified_df['generated_caption'] = modified_df['generated_caption'].str.replace(r'\baraf\w*\b', '', regex=True)

# Remove leading and trailing whitespace from the "generated_caption" column
modified_df['generated_caption'] = modified_df['generated_caption'].str.strip()

# Save the modified DataFrame to the same CSV file
modified_df.to_csv("/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/train/train_class_1_2_3_5_captioned.csv", index=False)

# Count the number of updated rows
updated_rows_count = (original_df['generated_caption'] != modified_df['generated_caption']).sum()

print(f"Number of updated rows: {updated_rows_count}")

Number of updated rows: 14525


------------------

# After fake generation

After generating all the fakes images that were missing, i need to merge the class_0_to_be_generated and the class_0_already_generated, and the same for class 1,2,3,5

Starting by adding the "fake_id" column to the train_class_0_to_be_generated csv

In [7]:
import pandas as pd

df = pd.read_csv("/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/train/train_class_0_to_be_generated.csv")

# new column 'fake_id'
df['fake_id'] = ''

# assign values to 'fake_id' based on row index intervals
df.loc[df.index < 17650, 'fake_id'] = 'SD_fake_' + df['id']
df.loc[(df.index >= 17650) & (df.index < 35606), 'fake_id'] = 'DL_fake_' + df['id']
df.loc[df.index >= 35606, 'fake_id'] = 'GL_fake_' + df['id']

# 'fake_id' column next to 'id' column
df.insert(df.columns.get_loc('id') + 1, 'fake_id', df.pop('fake_id'))

df.to_csv('/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/train/train_class_0_final.csv', index=False)

In [1]:
import pandas as pd

df = pd.read_csv("/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/train/train_class_1_2_3_5_to_be_generated.csv")

# new column 'fake_id'
df['fake_id'] = ''

# assign values to 'fake_id' based on row index intervals
df.loc[df.index < 12350, 'fake_id'] = 'SD_fake_' + df['id']
df.loc[(df.index >= 12350) & (df.index < 24394), 'fake_id'] = 'DL_fake_' + df['id']
df.loc[df.index >= 24394, 'fake_id'] = 'GL_fake_' + df['id']

# 'fake_id' column next to 'id' column
df.insert(df.columns.get_loc('id') + 1, 'fake_id', df.pop('fake_id'))

df.to_csv('/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/train/train_class_1_2_3_5_final.csv', index=False)

set the column pristine_image = 0 and generated_image = 1 for every row in train_class_0_final csv

In [8]:
df = pd.read_csv('csv/train/train_class_0_final.csv')

# Swap the values of the two columns
df['pristine_image'], df['generated_image'] = df['generated_image'], df['pristine_image']

df.to_csv("csv/train/train_class_0_final.csv", index=False)

In [2]:
df = pd.read_csv('csv/train/train_class_1_2_3_5_final.csv')

# Swap the values of the two columns
df['pristine_image'], df['generated_image'] = df['generated_image'], df['pristine_image']

df.to_csv("csv/train/train_class_1_2_3_5_final.csv", index=False)

Now i need to add the column "generated_caption"

In [25]:
import pandas as pd

df_a = pd.read_csv("csv/train/train_class_0_final.csv")
df_b = pd.read_csv("csv/train/train_class_0_captioned.csv")

# Perform inner join on 'id' column
df_c = pd.merge(df_a, df_b, on='id', how='inner')

# Rename 'original_caption_x' to 'original_caption'
df_c = df_c.rename(columns={'original_caption_x': 'original_caption'})

df_c = df_c[['id', 'fake_id', 'author', 'original_caption', 'generated_caption', 'num_comments', '6_way_label', 'pristine_image', 'generated_image', 'real_text', 'fakenews_text']]

df_c.to_csv("csv/train/train_class_0_final.csv", index=False)
df_c.head()

,id,fake_id,author,original_caption,generated_caption,num_comments,6_way_label,pristine_image,generated_image,real_text,fakenews_text
0,dhr269,SD_fake_dhr269,bico89,my bill was,someone is holding a receipt and a box of cheese,10.0,0,0,1,1,0
1,3gb4sc,SD_fake_3gb4sc,landofthemolotovia,quebec considering removing nword from place n...,there is a bear that is walking across the water,19.0,0,0,1,1,0
2,1wkhcb,SD_fake_1wkhcb,Zjw0115,dramatic baseball catcher,baseball player catching a ball in a stadium,6.0,0,0,1,1,0
3,bt0p1w,SD_fake_bt0p1w,tx69er,this mcdonalds uses aol for its corporate email,a close up of a receipt with a price tag on it,3.0,0,0,1,1,0
4,2yfbtu,SD_fake_2yfbtu,ProfessorFartdust,fan and katy perry,woman with a pink and blue hair and a man with...,5.0,0,0,1,1,0


In [3]:
import pandas as pd

df_a = pd.read_csv("csv/train/train_class_1_2_3_5_final.csv")
df_b = pd.read_csv("csv/train/train_class_1_2_3_5_captioned.csv")

# Perform inner join on 'id' column
df_c = pd.merge(df_a, df_b, on='id', how='inner')

# Rename 'original_caption_x' to 'original_caption'
df_c = df_c.rename(columns={'original_caption_x': 'original_caption'})

df_c = df_c[['id', 'fake_id', 'author', 'original_caption', 'generated_caption', 'num_comments', '6_way_label', 'pristine_image', 'generated_image', 'real_text', 'fakenews_text']]

df_c.to_csv("csv/train/train_class_1_2_3_5_final.csv", index=False)
df_c.head()

,id,fake_id,author,original_caption,generated_caption,num_comments,6_way_label,pristine_image,generated_image,real_text,fakenews_text
0,110klp,SD_fake_110klp,kopiikat,look at all the people having fun down there,crowd of people at a concert with their hands ...,0.0,2,0,1,0,1
1,8jthnp,SD_fake_8jthnp,Yare_Daze,zenith hereby unseen higher places,there is a poster of a man riding a motorcycle...,0.0,1,0,1,0,1
2,77x9zc,SD_fake_77x9zc,NikKerk,rare footage of a british soldier using a flam...,there is a black and white photo of a person i...,1.0,2,0,1,0,1
3,67yaae,SD_fake_67yaae,shittyusernamee,years ago i was burning an sat book for vengea...,book with a burning skull on it sitting on the...,5.0,2,0,1,0,1
4,95hg84,SD_fake_95hg84,politics_SS,transgender troops say trump has taken few ste...,a close up of a brick wall with a pink tongue ...,20.0,3,0,1,0,1


finally merge train_class_0_final + train_class_0_already_generated + train_class_1_2_3_5_final + train_class_1_2_3_5_already_generated to create train_final csv containing all the 192.935 generated images.

firstly drop the column class from train_class_0_already_generated and train_class_1_2_3_5_already_generated to make all the csv equal

In [6]:
df = pd.read_csv("/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/train/train_class_1_2_3_5_already_generated.csv")

# Drop the "class" column
column_to_drop = 'class'
df = df.drop(columns=[column_to_drop])

# Save the modified DataFrame back to a new CSV file
df.to_csv("/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/train/train_class_1_2_3_5_already_generated.csv", index=False)

and finally make the union of the df

In [7]:
import pandas as pd

# Read CSV files
df1 = pd.read_csv("csv/train/train_class_0_final.csv")
df3 = pd.read_csv("csv/train/train_class_0_already_generated.csv")
df2 = pd.read_csv("csv/train/train_class_1_2_3_5_final.csv")
df4 = pd.read_csv("csv/train/train_class_1_2_3_5_already_generated.csv")

# Concatenate DataFrames
df_union = pd.concat([df1, df2, df3, df4], ignore_index=True)

df_union.to_csv("csv/train/train_generated_final.csv", index=False)
print(len(df_union))

192936


Print out the cardinalities of the classes with the generation methods associated

In [8]:
import pandas as pd

df = pd.read_csv('csv/train/train_generated_final.csv')

# Group the DataFrame by '6_way_label'
grouped = df.groupby('6_way_label')

# Iterate over each group
for label, group in grouped:
    # Filter the group where 'id' column starts with "SD", "GL", or "DL"
    sd_count = len(group[group['fake_id'].str.startswith('SD')])
    gl_count = len(group[group['fake_id'].str.startswith('GL')])
    dl_count = len(group[group['fake_id'].str.startswith('DL')])
    
    # Print counts for each value of '6_way_label'
    print(f"Counts for '6_way_label' {label}:")
    print(f"Starts with 'SD': {sd_count}")
    print(f"Starts with 'GL': {gl_count}")
    print(f"Starts with 'DL': {dl_count}")
    print()

Counts for '6_way_label' 0:
Starts with 'SD': 40000
Starts with 'GL': 27549
Starts with 'DL': 40000

Counts for '6_way_label' 1:
Starts with 'SD': 5779
Starts with 'GL': 4827
Starts with 'DL': 5777

Counts for '6_way_label' 2:
Starts with 'SD': 18501
Starts with 'GL': 15744
Starts with 'DL': 18460

Counts for '6_way_label' 3:
Starts with 'SD': 1985
Starts with 'GL': 1668
Starts with 'DL': 2013

Counts for '6_way_label' 5:
Starts with 'SD': 3735
Starts with 'GL': 3148
Starts with 'DL': 3750



Now i need to merge validation_class_0_pristine and train_class_1_2_3_5_pristine to make train_pristine_final csv

In [1]:
import pandas as pd

df1 = pd.read_csv("csv/train/train_class_0_pristine.csv")
df2 = pd.read_csv("csv/train/train_class_1_2_3_5_pristine.csv")

# Concatenate DataFrames
df_union = pd.concat([df1, df2], ignore_index=True)

df_union.to_csv("csv/train/train_pristine_final.csv", index=False)
print(len(df_union))

192935


Finally i need to merge the train_pristine_final with train_generated_final to generated train csv

In [2]:
import pandas as pd

df_a = pd.read_csv('csv/train/train_generated_final.csv')
df_b = pd.read_csv('csv/train/train_pristine_final.csv')

# Add missing columns from A to B with empty values
for column in df_a.columns:
    if column not in df_b.columns:
        df_b[column] = ''

# Concatenate dataframes
df_c = pd.concat([df_a, df_b])

print(len(df_c))
df_c.to_csv('csv/train/train.csv', index=False)

385871


Show that half of them is generated, so the other half is pristine

In [4]:
import pandas as pd

df = pd.read_csv('csv/train/train.csv')

# Filter rows where the "fake_id" column starts with "SD", "DL", or "GL", or is empty
filtered_df = df[df['fake_id'].apply(lambda x: str(x).startswith(('SD', 'DL', 'GL')))]

print("Number of rows where fake_id starts with 'SD', 'DL', or 'GL', or is empty:", len(filtered_df))

/tmp/ipykernel_1808720/1061100489.py:3: DtypeWarning: Columns (1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('csv/train/train.csv')


Number of rows where fake_id starts with 'SD', 'DL', or 'GL', or is empty: 192936


In [2]:
import os
import pandas as pd

df = pd.read_csv("csv/train/train.csv")

selected_ids = []

for index, row in df.iterrows():
    if row['pristine_image'] == 1:
        selected_ids.append(row['id'])
    else:
        selected_ids.append(row['fake_id'])

# Get a list of all the image filenames in the folder (without extensions)
folder_path = "/home/enriconello/DeepFakeDetection/dataset_new/train"
folder_images = set(os.path.splitext(filename)[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = set(selected_ids).intersection(folder_images)
num_matching_images = len(matching_images)

print("Number of matching images:", num_matching_images)

Number of matching images: 385871


Set column "class" = 0 if column "fake_id" is not empty, = 1 otherwise

In [2]:
import pandas as pd

df = pd.read_csv("csv/train/train.csv")

df['class'] = df['fake_id'].apply(lambda x: 0 if pd.isnull(x) else 1)

df.to_csv("csv/train/train.csv", index=False)

/tmp/ipykernel_1663797/943676688.py:3: DtypeWarning: Columns (1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("csv/train/train.csv")


shuffle the csv row order

In [3]:
import pandas as pd

df = pd.read_csv('/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/train/train.csv')

# Shuffle the rows of the DataFrame
df_shuffled = df.sample(frac=1).reset_index(drop=True)

df_shuffled.to_csv('/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/train/train.csv', index=False)

/tmp/ipykernel_1663797/1934373135.py:3: DtypeWarning: Columns (1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/train/train.csv')


-----------

# Experiment Train Biased Type 2

Reconstruct the csv for train, deleting all the rows of generated samples from class 0 (type 1). Then it will contain only pristine class 0, pristine + generated class 1,2,3,5

In [4]:
import pandas as pd

df = pd.read_csv("C:/Users/nello/Documents/vscode_projects/Thesis/5_biased_detection/csv/train/train.csv")
print("len before: "+str(len(df)))

filtered_df = df[~((df['generated_image'] == 1) & (df['real_text'] == 1))]

filtered_df.to_csv("C:/Users/nello/Documents/vscode_projects/Thesis/5_biased_detection/csv/train/train_biased_2.csv", index=False)
print("len after: "+str(len(filtered_df)))

len before: 385871
len after: 278322


Add a column "type" that will be = 1 if the 6_way_label == 0, else will be type = 2

In [ ]:
import pandas as pd

df = pd.read_csv('C:/Users/nello/Documents/vscode_projects/Thesis/5_biased_detection/csv/train/train_biased_2.csv')

df['type'] = df['6_way_label'].apply(lambda x: 1 if x == 0 else 2)

df.to_csv('C:/Users/nello/Documents/vscode_projects/Thesis/5_biased_detection/csv/train/train_biased_2.csv', index=False)